In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data_from_dir = 'C:/Users/young/Desktop/R/cats-dogs'
base_dir = base_dir='C:/Users/young/Desktop/R/kaggle_cat'

train_dir='C:/Users/young/Desktop/R/kaggle_cat/train'
val_dir='C:/Users/young/Desktop/R/kaggle_cat/val'
test_dir='C:/Users/young/Desktop/R/kaggle_cat/test'

train_cats_dir=f'C:/Users/young/Desktop/R/kaggle_cat/train/cats'
train_dogs_dir='C:/Users/young/Desktop/R/kaggle_cat/train/dogs'
val_cats_dir='C:/Users/young/Desktop/R/kaggle_cat/val/cats'
val_dogs_dir='C:/Users/young/Desktop/R/kaggle_cat/val/dogs'
test_cats_dir='C:/Users/young/Desktop/R/kaggle_cat/test/cats'
test_dogs_dir='C:/Users/young/Desktop/R/kaggle_cat/test/dogs'


In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1/255,
                             zoom_range= 0.2)
data_gen = ImageDataGenerator(rescale = 1/255)

train_gen = train_data_gen.flow_from_directory(train_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='binary')
val_gen = data_gen.flow_from_directory(val_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='binary')
test_gen = data_gen.flow_from_directory(test_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
from keras.applications import VGG16

conv_vgg16 =VGG16(weights='imagenet', include_top=False,input_shape=(150,150,3))
conv_vgg16.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
bloc

In [5]:
for layer in conv_vgg16.layers:
    layer.trainable = False

In [6]:
from keras import models, layers, optimizers

model = models.Sequential()

model.add(conv_vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='binary_crossentropy',
              metrics=['acc'])


              

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = 100,
                              epochs = 10,
                              validation_data = val_gen,
                              validation_steps = 50
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 15s 150ms/step - loss: 0.6181 - acc: 0.6765 - val_loss: 0.5202 - val_acc: 0.7990
Epoch 2/10
100/100 [==============================] - 12s 119ms/step - loss: 0.4753 - acc: 0.8130 - val_loss: 0.4192 - val_acc: 0.8420
Epoch 3/10
100/100 [==============================] - 12s 118ms/step - loss: 0.4037 - acc: 0.8375 - val_loss: 0.3663 - val_acc: 0.8690
Epoch 4/10
100/100 [==============================] - 12s 119ms/step - loss: 0.3612 - acc: 0.8615 - val_loss: 0.3370 - val_acc: 0.8700
Epoch 5/10
100/100 [==============================] - 12s 119ms/step - loss: 0.3290 - acc: 0.8750 - val_loss: 0.3162 - val_acc: 0.8900
Epoch 6/10
100/100 [==============================] - 12s 118ms/step - loss: 0.3110 - acc: 0.8795 - val_loss: 0.2967 - val_acc: 0.8900
Epoch 7/10
100/100 [==============================] - 12s 118ms/step - loss: 0.2891 - acc: 0.8900 - val_loss: 0.2877 - val_acc: 0.8850
Epoch 8

In [8]:
# fine tuning

for i, layer in enumerate(conv_vgg16.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [9]:
for layer in conv_vgg16.layers[:15]:
    layer.trainable = False
for layer in conv_vgg16.layers[15:]:
    layer.trainable = True

In [10]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-6),
             loss='binary_crossentropy',
             metrics=['acc'])


In [11]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = 100,
                              epochs = 10,
                              validation_data = val_gen,
                              validation_steps = 50
)

Epoch 1/10
100/100 [==============================] - 13s 129ms/step - loss: 0.2437 - acc: 0.9120 - val_loss: 0.2563 - val_acc: 0.8960
Epoch 2/10
100/100 [==============================] - 12s 118ms/step - loss: 0.2219 - acc: 0.9180 - val_loss: 0.2466 - val_acc: 0.8930
Epoch 3/10
100/100 [==============================] - 12s 117ms/step - loss: 0.2012 - acc: 0.9260 - val_loss: 0.2320 - val_acc: 0.9040
Epoch 4/10
100/100 [==============================] - 12s 118ms/step - loss: 0.2039 - acc: 0.9305 - val_loss: 0.2110 - val_acc: 0.9110
Epoch 5/10
100/100 [==============================] - 12s 117ms/step - loss: 0.1933 - acc: 0.9280 - val_loss: 0.2449 - val_acc: 0.89601s - loss
Epoch 6/10
100/100 [==============================] - 12s 117ms/step - loss: 0.1736 - acc: 0.9380 - val_loss: 0.2156 - val_acc: 0.9160
Epoch 7/10
100/100 [==============================] - 12s 117ms/step - loss: 0.1856 - acc: 0.9275 - val_loss: 0.2087 - val_acc: 0.9070
Epoch 8/10
100/100 [==========================